# Projeto #2 - Classificador supervisionado

Antes de começar, leia as [Instruções](https://github.com/thvmm/pos-ds-ia/tree/master/projeto_2#instru%C3%A7%C3%B5es) e os [Critérios de Avaliação](https://github.com/thvmm/pos-ds-ia/tree/master/projeto_2#crit%C3%A9rios-de-avalia%C3%A7%C3%A3o)


### 1) Qual a base escolhida?

In [131]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier

# Base Escolhida: Forest covertypes - https://archive.ics.uci.edu/ml/datasets/Covertype

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
arquivo_base = '/content/drive/My Drive/covtype.csv'

df = pd.read_csv(arquivo_base, sep=',')
df.head()

Mounted at /content/drive


,Elevation,Aspect,slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Wilderness_Area_4,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_1.1,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


### 2) **(10%)** Pré-processamento: entendimento do conjunto de dados

In [0]:
# Quais são as minhas features?

# Elevation, 
# Aspect, 
# Slope, 
# Horizontal Distance to Hydrology, 
# Vertical Distance to Hydrology, 
# Horizontal Distance to Roadways, 
# Hillshade at 9am, 
# Hillshade at Noon, 
# Hillshade at 3pm, 
# Horizontal Distance to fire points,
# Wilderness Area (4 tipos), 
# Soil Type (40 tipos), 
# Cover_Type (o que queremos classificar)

# Quais são as minhas classes?
# 1: Spruce/Fir
# 2: Lodgepole Pine
# 3: Ponderosa Pine
# 4: Cottonwood/Willow
# 5: Aspen
# 6: Douglas-fir
# 7: Krummholz

In [0]:
#Como estão distribuídas as minhas classes?
df.groupby('Cover_Type').size()


Cover_Type
1    211840
2    283301
3     35754
4      2747
5      9493
6     17367
7     20510
dtype: int64

In [133]:
# Checagem se os valores estão dentro de um limite permitido ou razoável.
df.describe()


,Elevation,Aspect,slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Wilderness_Area_4,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_1.1,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,0.448865,0.051434,0.436074,0.063627,0.005217,0.012952,0.008301,0.021335,0.002749,0.011316,0.000181,0.000308,0.001974,0.056168,0.021359,0.051584,0.030001,0.001031,0.000005,0.004897,0.005890,0.003268,0.006921,0.015936,0.001442,0.057439,0.099399,0.036622,0.000816,0.004456,0.001869,0.001628,0.198356,0.051927,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,0.497379,0.220882,0.495897,0.244087,0.072039,0.113066,0.090731,0.144499,0.052356,0.105775,0.013442,0.017550,0.044387,0.230245,0.144579,0.221186,0.170590,0.032092,0.002272,0.069804,0.076518,0.057077,0.082902,0.125228,0.037950,0.232681,0.299197,0.187833,0.028551,0.066605,0.043193,0.040318,0.398762,0.221879,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

### 3) **(80%)** Nos blocos seguintes implemente seus classificadores (serão implementados 2 métodos diferentes).

#### 3.1) Qual método escolhido?
> Naive Bayes

#### 3.2) **(10%)** Baseline - Implemente seu classificador da forma mais simples possível para esse ser seu baseline

In [0]:
# Dividindo o dataset em treino e teste. Meu y é a coluna que queremos classificar, cover_type
y = df['Cover_Type']
df = df.drop(['Cover_Type'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(df,
                                                    y, 
                                                    test_size=0.25)

In [0]:
classificador = GaussianNB()
classificador.fit(X_train, y_train)

y_pred = classificador.predict(X_test)


In [136]:
acuracia = accuracy_score(y_pred, y_test)

print("Acurácia Inicial: " + str(acuracia))

Acurácia Inicial: 0.4598115013115048


#### 3.3) **(20%)** Versão 1 - O que podemos fazer para melhorar nosso baseline? Aplique técnicas como redução de dimensionalidade, normalização ou outras. Compare os resultados.

In [137]:
# Redução de dimensionalidade usando a técnica de análise de variância
sel = VarianceThreshold(threshold=0.05)
df_variance_threshold = sel.fit_transform(df)
df_variance_threshold[0:5,:]

array([[2596,   51,    3,  258,    0,  510,  221,  232,  148, 6279,    1,
           0,    0,    0,    0,    0,    1,    0,    0],
       [2590,   56,    2,  212,   -6,  390,  220,  235,  151, 6225,    1,
           0,    0,    0,    0,    0,    1,    0,    0],
       [2804,  139,    9,  268,   65, 3180,  234,  238,  135, 6121,    1,
           0,    0,    0,    0,    0,    0,    0,    0],
       [2785,  155,   18,  242,  118, 3090,  238,  238,  122, 6211,    1,
           0,    0,    0,    0,    0,    0,    0,    0],
       [2595,   45,    2,  153,   -1,  391,  220,  234,  150, 6172,    1,
           0,    0,    0,    0,    0,    1,    0,    0]])

In [138]:
# De 54 features, agora temos 19

print(df.shape)
print(df_variance_threshold.shape)
print(df_variance_threshold)

(581012, 54)
(581012, 19)
[[2596   51    3 ...    1    0    0]
 [2590   56    2 ...    1    0    0]
 [2804  139    9 ...    0    0    0]
 ...
 [2386  159   17 ...    0    0    0]
 [2384  170   15 ...    0    0    0]
 [2383  165   13 ...    0    0    0]]


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_variance_threshold,
                                                    y, 
                                                    test_size=0.25)

classificador.fit(X_train, y_train)

y_pred = classificador.predict(X_test)

In [140]:
acuracia = accuracy_score(y_pred, y_test)
print("Acurácia após Redução de Dimensionalidade: " + str(acuracia))

Acurácia após Redução de Dimensionalidade: 0.5055936882542873


#### 3.4) **(10%)** Tunning - Agora que temos um resultado promissor, vamos tentar melhorar o resultado alterando um ou mais hiper-parametro. Compare os resultados.

In [142]:
# Tunning



Acuracia de 0.5065437546900925


#### 3.5) Qual método escolhido?

Random Forest

#### 3.6) **(10%)** Baseline - Implemente seu classificador da forma mais simples possível para esse ser seu baseline

In [144]:

X_train, X_test, y_train, y_test = train_test_split(df,
                                                    y, 
                                                    test_size=0.25)

classificadorRF = RandomForestClassifier(max_depth=2, random_state=0)
classificadorRF.fit(X_train, y_train)

y_predRF = classificadorRF.predict(X_test)

[2.01095364e-01 3.47401242e-03 3.92997289e-03 1.87700140e-03
 1.04959844e-03 4.35290163e-02 2.46060284e-03 7.53109679e-04
 8.99628560e-04 2.96119354e-02 3.16823989e-02 1.90104902e-02
 1.50586833e-03 1.89394655e-01 3.95366444e-03 7.62508678e-03
 5.46050700e-04 4.34186982e-02 1.16257296e-03 1.23450205e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.77672873e-02
 7.89524013e-04 7.92140908e-02 6.50352038e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00180196e-01
 2.23592823e-02 1.29076511e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.93324350e-02 1.63229129e-03
 7.69039601e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.76867228e-02
 4.19540570e-02 1.30498638e-02]


In [145]:
acuracia = accuracy_score(y_predRF, y_test)
print("Acurácia de "+str(acuracia))

Acurácia de 0.5675338891451468


#### 3.7) **(20%)** Versão 1 - O que podemos fazer para melhorar nosso baseline? Aplique técnicas como redução de dimensionalidade, normalização ou outras. Compare os resultados.

In [0]:
# aplicando o treino no df_variance_threshold onde foi aplicada a redução da dimensionalidade

X_train, X_test, y_train, y_test = train_test_split(df_variance_threshold,
                                                    y, 
                                                    test_size=0.25)

classificadorRF = RandomForestClassifier(max_depth=2, random_state=0)
classificadorRF.fit(X_train, y_train)

y_predRF = classificadorRF.predict(X_test)

In [147]:
acuracia = accuracy_score(y_predRF, y_test)
print("Acurácia de "+str(acuracia))

Acurácia de 0.6479521937584766


#### 3.8) **(10%)** Tunning - Agora que temos um resultado promissor, vamos tentar melhorar o resultado alterando um ou mais hiper-parametro. Compare os resultados.

In [148]:
classificadorRF = RandomForestClassifier(n_estimators=100, max_depth=10, max_features=0.3, bootstrap=True, min_samples_leaf=1, n_jobs=-1)
classificadorRF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=0.3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [149]:
y_predRF = classificadorRF.predict(X_test)
acuracia = accuracy_score(y_predRF, y_test)
print("Acurácia de "+str(acuracia))

Acurácia de 0.7788479411784954


### 5) **(10%)** Conclusões

*Compare seus resultados. Imaginando que sua solução fosse para produção, qual deles você escolheria? Por que? Quais os riscos você enxerga? O que recomendaria de próximos passos para melhorar os resultados?*